In [15]:
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import pandas as pd
import os
import json
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, classification_report
import torch.nn.functional as F
import re
import shutil
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import torch
from flask import Flask, request
from flask import render_template
from flask_ngrok import run_with_ngrok
ws_driver = CkipWordSegmenter(level=3,device=0)
import random

In [16]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(20)

In [17]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, PackedSequence


class HierarchialAttentionNetwork(nn.Module):
    def __init__(self, n_classes, vocab_size, emb_size, word_rnn_size, sentence_rnn_size, word_rnn_layers,
                 sentence_rnn_layers, word_att_size, sentence_att_size, dropout=0.5):
        super(HierarchialAttentionNetwork, self).__init__()
        self.sentence_attention = SentenceAttention(vocab_size, emb_size, word_rnn_size, sentence_rnn_size,
                                                    word_rnn_layers, sentence_rnn_layers, word_att_size,
                                                    sentence_att_size, dropout)

        self.fc = nn.Linear(2 * sentence_rnn_size, n_classes)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, documents, sentences_per_document, words_per_sentence):
        document_embeddings, word_alphas, sentence_alphas = self.sentence_attention(documents, sentences_per_document,
                                                                                    words_per_sentence)  
        scores = self.sigmoid(self.fc(self.dropout(document_embeddings)) ) 
        
        return scores, word_alphas, sentence_alphas


class SentenceAttention(nn.Module):
    def __init__(self, vocab_size, emb_size, word_rnn_size, sentence_rnn_size, word_rnn_layers, sentence_rnn_layers,
                 word_att_size, sentence_att_size, dropout):
        super(SentenceAttention, self).__init__()

        self.word_attention = WordAttention(vocab_size, emb_size, word_rnn_size, word_rnn_layers, word_att_size,
                                            dropout)

        self.sentence_rnn = nn.GRU(2 * word_rnn_size, sentence_rnn_size, num_layers=sentence_rnn_layers,
                                   bidirectional=True, dropout=dropout, batch_first=True)

        self.sentence_attention = nn.Linear(2 * sentence_rnn_size, sentence_att_size)

        self.sentence_context_vector = nn.Linear(sentence_att_size, 1,
                                                 bias=False) 

        self.dropout = nn.Dropout(dropout)

    def forward(self, documents, sentences_per_document, words_per_sentence):
        packed_sentences = pack_padded_sequence(documents,
                                                lengths=sentences_per_document.tolist(),
                                                batch_first=True,
                                                enforce_sorted=False) 
        packed_words_per_sentence = pack_padded_sequence(words_per_sentence,
                                                         lengths=sentences_per_document.tolist(),
                                                         batch_first=True,
                                                         enforce_sorted=False)  
        sentences, word_alphas = self.word_attention(packed_sentences.data,
                                                     packed_words_per_sentence.data)  
        
        sentences = self.dropout(sentences)
        
        packed_sentences, _ = self.sentence_rnn(PackedSequence(data=sentences,
                                                               batch_sizes=packed_sentences.batch_sizes,
                                                               sorted_indices=packed_sentences.sorted_indices,
                                                               unsorted_indices=packed_sentences.unsorted_indices))  
        att_s = self.sentence_attention(packed_sentences.data)  
        att_s = torch.tanh(att_s)  
        
        att_s = self.sentence_context_vector(att_s).squeeze(1)  

        max_value = att_s.max()  
        att_s = torch.exp(att_s - max_value) 

        att_s, _ = pad_packed_sequence(PackedSequence(data=att_s,
                                                      batch_sizes=packed_sentences.batch_sizes,
                                                      sorted_indices=packed_sentences.sorted_indices,
                                                      unsorted_indices=packed_sentences.unsorted_indices),
                                       batch_first=True)  # (n_documents, max(sentences_per_document))

  
        sentence_alphas = att_s / torch.sum(att_s, dim=1, keepdim=True)

        documents, _ = pad_packed_sequence(packed_sentences,
                                           batch_first=True)  
        documents = documents * sentence_alphas.unsqueeze(2)  
        documents = documents.sum(dim=1)
        word_alphas, _ = pad_packed_sequence(PackedSequence(data=word_alphas,
                                                            batch_sizes=packed_sentences.batch_sizes,
                                                            sorted_indices=packed_sentences.sorted_indices,
                                                            unsorted_indices=packed_sentences.unsorted_indices),
                                             batch_first=True) 
        return documents, word_alphas, sentence_alphas


class WordAttention(nn.Module):
    def __init__(self, vocab_size, emb_size, word_rnn_size, word_rnn_layers, word_att_size, dropout):
        super(WordAttention, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, emb_size)
        self.word_rnn = nn.GRU(emb_size, word_rnn_size, num_layers=word_rnn_layers, bidirectional=True,
                               dropout=dropout, batch_first=True)

        self.word_attention = nn.Linear(2 * word_rnn_size, word_att_size)
        self.word_context_vector = nn.Linear(word_att_size, 1, bias=False)

        self.dropout = nn.Dropout(dropout)

    def init_embeddings(self, embeddings):
        self.embeddings.weight = nn.Parameter(embeddings)

    def fine_tune_embeddings(self, fine_tune=False):
        for p in self.embeddings.parameters():
            p.requires_grad = fine_tune

    def forward(self, sentences, words_per_sentence):
        sentences = self.dropout(self.embeddings(sentences))
        packed_words = pack_padded_sequence(sentences,
                                            lengths=words_per_sentence.tolist(),
                                            batch_first=True,
                                            enforce_sorted=False) 
        # a PackedSequence object, where 'data' is the flattened words (n_words, word_emb)

        packed_words, _ = self.word_rnn(packed_words)  
        att_w = self.word_attention(packed_words.data)
        att_w = torch.tanh(att_w) 
        att_w = self.word_context_vector(att_w).squeeze(1)  # (n_words)

        max_value = att_w.max()  
        att_w = torch.exp(att_w - max_value)  
        
        att_w, _ = pad_packed_sequence(PackedSequence(data=att_w,batch_sizes=packed_words.batch_sizes,
                                    sorted_indices=packed_words.sorted_indices,
                                    unsorted_indices=packed_words.unsorted_indices),
                                       batch_first=True)
        word_alphas = att_w / torch.sum(att_w, dim=1, keepdim=True) 

        sentences, _ = pad_packed_sequence(packed_words,
                                           batch_first=True)  

        sentences = sentences * word_alphas.unsqueeze(2) 
        sentences = sentences.sum(dim=1) 

        return sentences, word_alphas

In [18]:
with open('fin_word.json', 'r', encoding='utf-8') as f:
    word2idx = json.load(f)
with open('fin_tag.json', 'r', encoding='utf-8') as f:
    tag2idx = json.load(f)
word2idx['<pad>'] = 0
idx2word = {word2idx[w]:w for w in word2idx}
idx2tag = {tag2idx[t]:t for t in tag2idx}
def clean_str(string):
    string = re.sub(r"\u3000", "", string)
    string = re.sub(r"\s{1,}", "", string)
    string = re.sub(r"\r", "", string)
    string = re.sub(r"\n", "", string)
    string = re.sub(r"【[^【】]+】","",string)
    string = re.sub(r"（[^（）]+）","",string)
    string = re.sub(r"「[^「」]+」","",string)
    string = re.sub(r"\([^\(\)]+\)","",string)
    if '審判長' in string:
        string = string.split('審判長')[0]
    else:
        string = string.split('書記官')[0]
    return string.strip()
def collate_fn(bs_doc , bs_dlen , bs_slen , word_pad_idx,
               word_limit = 40,sentence_limit=80):
    max_sent_len = min(max(bs_slen) ,word_limit )
    min_doc_len = min(sentence_limit , bs_dlen)
    pad_doc = []
    for sent in bs_doc:
        if len(sent) > max_sent_len:
            pad_doc.append(sent[:max_sent_len])
        else:
            pad_doc.append(sent + (max_sent_len-len(sent))*[word_pad_idx])
    if len(pad_doc) > min_doc_len:
        pad_doc = pad_doc[:min_doc_len]
    bs_slen_p = []
    for i in bs_slen:
        if i > max_sent_len:
            bs_slen_p.append(max_sent_len)
            continue
        bs_slen_p.append(i)
    return torch.LongTensor([pad_doc]) ,torch.tensor([min_doc_len]), torch.LongTensor([bs_slen_p])
def inp_len(x):
    if len(x) > 0:
        return x

In [19]:
def Do_all(text,pad_idx):
    process_text = clean_str(text)
    doc = []
    for t in process_text.split('，'):
        if '。' in t:
            doc.extend(t.split('。'))
        else:
            doc.append(t)
    doc = list(filter(inp_len,doc))
    docs = [doc]
    ws = ws_driver(doc)
    doc_p = []
    s_len = []
    for voc in ws:
        doc_p.append(voc)
        s_len.append(len(voc))
    d_len = len(doc_p)
    docs_w2i = [list(map( lambda s:word2idx.get(s,word2idx['<unk>']) ,sent )) for sent in doc_p]
    documents,sentences_per_document,words_per_sentence = collate_fn(docs_w2i , d_len ,s_len,pad_idx)
    n_classes = len(tag2idx) 
    word_rnn_size = 100 
    sentence_rnn_size = 100 
    word_rnn_layers = 2  
    sentence_rnn_layers = 2  
    word_att_size = 200
    sentence_att_size = 200  
    dropout = 0.3
    lr = 1e-3
    with torch.no_grad():
        model = HierarchialAttentionNetwork(n_classes=n_classes,
                                            vocab_size=len(word2idx),
                                            emb_size=300,
                                            word_rnn_size=word_rnn_size,
                                            sentence_rnn_size=sentence_rnn_size,
                                            word_rnn_layers=word_rnn_layers,
                                            sentence_rnn_layers=sentence_rnn_layers,
                                            word_att_size=word_att_size,
                                            sentence_att_size=sentence_att_size,
                                            dropout=dropout)
        model.load_state_dict(torch.load('legaltech_han_model.pt'))
        scores, word_alphas, sentence_alphas = model(documents, sentences_per_document,
                                                                     words_per_sentence)
        predicts = ''
        for idx,s in enumerate(scores[0]):
            if s>0.5:
                predicts += idx2tag[idx] + '|'
        print(predicts)
    return predicts[:-1]

In [ ]:
TEMPLATE = 'templates'
STATIC = 'static'

app = Flask(__name__,template_folder=TEMPLATE,static_folder=STATIC)
run_with_ngrok(app)
input_text = ''
@app.route('/')
def index():
    return render_template('form.html')
@app.route('/submit', methods=['POST'])
def submit():
    input_text = request.form['text']
    if len(input_text) < 100:
        vis2 = 1
        s = 'Unable to send your message. Please fix errors then try again.'
        return render_template('form.html',vis2=vis2,input_text=input_text,s = s)
    elif len(input_text) > 100:
        vis1 = 1
        predicts = Do_all(input_text,word2idx['<pad>'])
        print(predicts)
        if predicts == '':
            predicts = '系統暫時未判讀出違規事宜'
        return render_template('form.html',vis1=vis1,input_text=input_text,s = predicts)

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Sep/2021 22:36:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2021 22:36:41] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:41] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:41] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:41] "GET /static/images/desktop_top-banner.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:41] "GET /static/images/banner-bg.png HTTP/1.1" 304 -


 * Running on http://40e2-140-127-114-22.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 32.31it/s]
127.0.0.1 - - [11/Sep/2021 22:36:57] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [11/Sep/2021 22:36:57] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:57] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:57] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:57] "GET /static/images/desktop_top-banner.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:36:57] "GET /static/images/banner-bg.png HTTP/1.1" 304 -


刑事訴訟法220|刑事訴訟法449 2|
刑事訴訟法220|刑事訴訟法449 2


...

Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.73it/s]
127.0.0.1 - - [11/Sep/2021 22:54:04] "POST /submit HTTP/1.1" 200 -


127.0.0.1 - - [11/Sep/2021 22:54:05] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:05] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:05] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:06] "GET /static/images/desktop_top-banner.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:06] "GET /static/images/banner-bg.png HTTP/1.1" 304 -
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 51.15it/s]
127.0.0.1 - - [11/Sep/2021 22:54:47] "POST /submit HTTP/1.1" 200 -


127.0.0.1 - - [11/Sep/2021 22:54:48] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:48] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:48] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:49] "GET /static/images/banner-bg.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:49] "GET /static/images/desktop_top-banner.png HTTP/1.1" 304 -
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 94.76it/s]
127.0.0.1 - - [11/Sep/2021 22:54:54] "POST /submit HTTP/1.1" 200 -


127.0.0.1 - - [11/Sep/2021 22:54:55] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:55] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:54:55] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:57:46] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:57:46] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:57:46] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 22:57:47] "GET /static/css/bootstrap/bootstrap.css.map HTTP/1.1" 200 -
Inference: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27.43it/s]
127.0.0.1 - - [11/Sep/2021 23:50:28] "POST /submit HTTP/1.1" 200 -


127.0.0.1 - - [11/Sep/2021 23:50:29] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 23:50:29] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 23:50:29] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 23:50:30] "GET /static/images/banner-bg.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Sep/2021 23:50:30] "GET /static/images/desktop_top-banner.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Sep/2021 14:56:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 14:56:09] "GET /static/css/bootstrap/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Sep/2021 14:56:09] "GET /static/css/nicepage.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Sep/2021 14:56:09] "GET /static/css/Page-1.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Sep/2021 14:56:10] "GET /static/images/desktop_top-banner.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Sep/2021 14:56:10] "GET /static/images/banner-bg.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Sep/2021 14:56:10] "GET /static/images/i